# Setup

In [1]:
!pip install -U -q transformers bitsandbytes accelerate sentence-transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.7 MB/s eta 0:00

In [17]:
from sentence_transformers import SentenceTransformer, util

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

from sklearn.cluster import KMeans

from datasets import load_dataset

from math import sqrt

## Create the sentence-embedder

In [3]:
class SentenceEncoder():
    def __init__(self, model='sentence-transformers/all-MiniLM-L12-v2', device='cuda'):
        self.model = SentenceTransformer(model, device)
        pass
    
    def encode(self, sentences, convert_to_tensor=True):
        return self.model.encode(sentences, convert_to_tensor=convert_to_tensor)

## Method to setup the summarizer pipeline

In [4]:
def create_summarizer_pipeline():
  model_name = "pszemraj/long-t5-tglobal-xl-16384-book-summary-8bit"
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
  summarizer = pipeline(task="summarization", model=model, tokenizer=tokenizer, device_map="auto", model_kwargs={"load_in_8bit": True})
  return summarizer

# Clustering Algorithms

## kmeans

In [11]:
def cluster_k_means(corpus, corpus_embeddings, num_clusters):
  clustering_model = KMeans(n_clusters=num_clusters)
  clustering_model.fit(corpus_embeddings)
  cluster_assignment = clustering_model.labels_
  clustered_sentences = [[] for i in range(num_clusters)]
  clustered_embeddings = [[] for i in range(num_clusters)]
  for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])
    clustered_embeddings[cluster_id].append(corpus_embeddings[sentence_id])
  return clustered_sentences, clustered_embeddings

# Quora Duplicate Questions Dataset

In [ ]:
def get_quora_questions_slice(slice_n):
  quora_dataset = load_dataset("quora", split="train")
  pass

In [6]:
slice_n = 10000
slice_quora_dataset = quora_dataset[:slice_n]

In [7]:
questions = []
for q in slice_quora_dataset['questions']:
  questions.append(q['text'][0])
  questions.append(q['text'][1])

In [8]:
st = SentenceEncoder()

In [10]:
embed_questions = st.encode(questions)

In [18]:
num_clusters = int(sqrt(len(questions)))

In [19]:
clustered_sentences, clustered_embeddings = cluster_k_means(corpus=questions,\
                                                            corpus_embeddings=embed_questions.cpu(),\
                                                            num_clusters=num_clusters)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [16]:
sqrt(len(questions))

141.4213562373095

In [20]:
clustered_sentences[0]

['What is the right etiquette for wishing a Jehovah Witness happy birthday?',
 'Who are the Rohingya Muslims?',
 'Why are there so many Christians in Kerala?',
 'How can I convince people to stop blaspheming against the Holy Spirit?',
 'Is circumcision allowed in Islam?',
 'Can a Muslim guy date a non Muslim girl? Is it based on how religious the individual is or are there other factors to it?',
 'What do non-Muslims expect from Muslims?',
 'Why did the Umayyad Caliphate fall? What could have been done to avoid his fall?',
 'History of Islam: Why did the Umayyad Caliphate invade Christian territory?',
 'Why is Persian word “ذليل” meaning ‘a Muslim’ translated incorrectly on Microsoft Translator? Is it intentional or careless?',
 'If the whole world were to follow only one religion, would it be more peaceful?',
 'What is the difference between Shia and Shiite?',
 'In what way is the Ismaili sect different from the other Shia groups?',
 'What is the appropriate way to wish a Muslim frien